<div class="alert alert-block alert-info">
<h2><center><strong> Participez à la conception d'une voiture autonome :Transformation </strong></center></h2>
        
</div>

<div class="alert alert-block alert-success">  
<strong>1.1 Importing des packages<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.spatial import distance
import os
import _pickle as cPickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# generate random integer values
from random import seed
from random import randint
seed(42)
from scipy.spatial import distance
from scipy import sparse

<div class="alert alert-block alert-danger">  
<h2><center><strong>2.Etude du jeu de données<a id='head-2'></a></strong></center></h2>
        
</div>

<div class="alert alert-block alert-success">  
<strong>2.1 Chargement du jeu de données<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [3]:

li = []
for dirname, _, filenames in os.walk('data/clicks'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        df = pd.read_csv(os.path.join(dirname, filename), index_col=None, header=0)
        li.append(df)

dfClicks = pd.concat(li, axis=0, ignore_index=True)
dfClicks.head(10)

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
5,2,1506825435299739,1506825435000,2,30970,1506827120575,4,1,17,1,24,2
6,3,1506825442704740,1506825442000,2,236065,1506827536942,4,3,2,1,21,1
7,3,1506825442704740,1506825442000,2,236294,1506827566942,4,3,2,1,21,1
8,4,1506825528135741,1506825528000,2,48915,1506826927593,4,1,17,1,17,1
9,4,1506825528135741,1506825528000,2,44488,1506826957593,4,1,17,1,17,1


In [4]:
dfSample = pd.read_csv('data/clicks_sample.csv', index_col=None, header=0)

In [5]:
users=pd.DataFrame({"user_id":dfClicks['user_id']})
sessions=pd.DataFrame({"session_id":dfClicks['session_id']})

In [6]:
articlesDf = pd.read_csv('data/articles_metadata.csv', index_col=None, header=0)
print(articlesDf.shape)
articlesDf.head()

(364047, 5)


,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


In [7]:
with open(r"data/articles_embeddings.pickle", "rb") as input_file:
    e = cPickle.load(input_file)

In [8]:
def getFiveArticles(e, userId):
    
    ee=e
    #get all articles read by user
    var= dfClicks.loc[dfClicks['user_id']==userId]['click_article_id'].tolist()
    #chose randomly one
    value = randint(0, len(var))
    #delete all read articles except the selected one( we do not want to offer user to read something he already read)
    for i in range(0, len(var)):
        if i != value:
            ee=np.delete(ee,[i],0)
    arr=[]
    
    #delecte selected article in the new matrix
    f=np.delete(ee,[value],0)
    #get 5 articles the most similar to the selected one
    for i in range(0,5):
        distances = distance.cdist([ee[value]], f, "cosine")[0]
        min_index = np.argmin(distances)
        f=np.delete(f,[min_index],0)
        #find corresponding matrix in original martix
        result = np.where(e == f[min_index])
        arr.append(result[0][0])
        
    return arr

# Content based filtering based on articles embeddings

#### selecting 5 closest articles using cosin distance

In [9]:
print(getFiveArticles(e, 92059))

[1415, 784, 785, 511, 936]


### Adding words count to article embeddings and selecting 5 closes articles

In [10]:
d=e
max=articlesDf['words_count'].max()
articlesDf['words_count']= articlesDf['words_count'].apply(lambda x: x/max)

d=np.append(d,np.reshape(articlesDf['words_count'].to_numpy(), newshape=(articlesDf['words_count'].shape[0],1)),axis=1)

d.shape

(364047, 251)

In [11]:
print(getFiveArticles(d, 92059))

[173, 219, 144, 9, 202]


In [12]:
n_users = dfClicks['user_id'].values.ravel()
n_users=pd.unique(n_users)
n_sessions = dfClicks['session_id'].values.ravel()
n_sessions=pd.unique(n_sessions)

## DNN Recommender system

Create target variable ( articles embeddings )

In [13]:
#Target Variable: articles
y= np.zeros((dfClicks.shape[0], 250))


for i in range(0,dfClicks.shape[0]):
    y[i]=e[[dfClicks.iloc[i,: ]['click_article_id']]]

#### Get reordered articles metadata dataframe that corresponds to sessions dataframe

On divise dfClicks en 10

In [14]:
from tqdm import tqdm
'''
#processed separately in batches
minValue = 0
maxValueInit = int(len(dfClicks)/100)
maxValue = int(len(dfClicks)/100)
for i in range(0,100) :
    
    articles_prepared= pd.DataFrame(columns=["article_id","category_id","created_at_ts","publisher_id","words_count"])
    if (i==99) :
        maxValue = len(dfClicks)
    print(minValue,maxValue)
    dfClicksSample = dfClicks[minValue:maxValue]

    for index, row in tqdm(dfClicksSample.iterrows(), total=len(dfClicksSample)):
        
        articles_prepared= articles_prepared.append(articlesDf.loc[row['click_article_id']])
    if i == 0 : 
        articles_prepared.to_csv(r'export_dataframe.csv', index = False, header=True)
    else :
        articles_prepared.to_csv(r'export_dataframe.csv', mode='a', header=False)
    minValue =  minValue + maxValueInit
    maxValue = maxValue + maxValueInit
    del(articles_prepared)
'''

'\n#processed separately in batches\nminValue = 0\nmaxValueInit = int(len(dfClicks)/100)\nmaxValue = int(len(dfClicks)/100)\nfor i in range(0,100) :\n    \n    articles_prepared= pd.DataFrame(columns=["article_id","category_id","created_at_ts","publisher_id","words_count"])\n    if (i==99) :\n        maxValue = len(dfClicks)\n    print(minValue,maxValue)\n    dfClicksSample = dfClicks[minValue:maxValue]\n\n    for index, row in tqdm(dfClicksSample.iterrows(), total=len(dfClicksSample)):\n        \n        articles_prepared= articles_prepared.append(articlesDf.loc[row[\'click_article_id\']])\n    if i == 0 : \n        articles_prepared.to_csv(r\'export_dataframe.csv\', index = False, header=True)\n    else :\n        articles_prepared.to_csv(r\'export_dataframe.csv\', mode=\'a\', header=False)\n    minValue =  minValue + maxValueInit\n    maxValue = maxValue + maxValueInit\n    del(articles_prepared)\n'

In [15]:
dfCategory = pd.read_csv('export_dataframe.csv', sep = ',', index_col=None, header=0)

In [16]:
dfCategory.head()

,article_id,category_id,created_at_ts,publisher_id,words_count
0,157541,281,1506800518000,0,280
1,68866,136,1506816482000,0,226
2,235840,375,1506807839000,0,159
3,96663,209,1506788025000,0,206
4,119592,247,1506784316000,0,239


#### DNN based on article category, word count, user,ategory_id words_count, user_id, date_created

In [17]:
n_cats = dfCategory['category_id'].values.ravel()
n_cats=pd.unique(dfCategory['category_id'])
n_users = users.values.ravel()
n_users=pd.unique(n_users)

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Multiply, LSTM, Dropout
from keras.models import Model
from keras.optimizers import SGD

embedding_size = 250

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users.shape[0],embedding_size, name="Users-Embedding")(user_input)
user_vec = Dense(embedding_size, activation='relu')(user_embedding)

cat_input = Input(shape=[1], name="Category-Input")
cat_embedding = Embedding(int(dfCategory['category_id'].max()), embedding_size, name="Catgory-Embedding")(cat_input)
cat_vec = Dense(embedding_size, activation='relu')(cat_embedding)

words_input = Input(shape=[1], name="Words-Input")
words_vec=Dense(embedding_size, activation='relu')(words_input)

date_input= Input(shape=(1,1),name="Date_Created-Input")
date_vec= LSTM(500, input_shape=(1,1))(date_input)
date_drop=Dropout(0.2)(date_vec)
date_dense= Dense(embedding_size, activation="relu")(date_drop)

prod = Multiply()([user_vec,cat_vec,words_vec,date_dense])

dense2= Dense(embedding_size, activation ="relu")(prod)
fin= Flatten()(dense2)
model = Model([user_input, cat_input,words_input, date_input], fin)
opt = SGD(learning_rate=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae', 'mse'] )
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Date_Created-Input (InputLayer) [(None, 1, 1)]       0                                            
__________________________________________________________________________________________________
User-Input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Category-Input (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 500)          1004000     Date_Created-Input[0][0]         
_______________________________________________________________________________________

In [18]:
import multiprocessing
_max=dfCategory['words_count'].max()
model.fit([users,
           dfCategory['category_id'],
           dfCategory['words_count'].apply(lambda x: x/_max),
           dfCategory['created_at_ts'].to_numpy().reshape( -1,1,1)] ,
           y, 
           epochs=10, 
           batch_size=128,  
           verbose=1,
           validation_split=0.2)

MemoryError: Unable to allocate 2.23 GiB for an array with shape (2390544, 250) and data type float32

In [ ]:
model.save('model.h5')

# Collaborative based filtering

https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b

On supprime les outliers , si un article a été cliqué < 10, on supprime

In [68]:
dfNbClicksByArticle = dfClicks.groupby(by="click_article_id")['user_id'].count()

In [71]:
dfClicks = pd.DataFrame(dfNbClicksByArticle, columns=["user_id"])

In [76]:
dfClicks[(dfClicks.user_id > 10)]

,user_id
click_article_id,
1843,11
1844,13
1873,30
1877,16
1932,42
...,...
363952,41
363967,79
363976,43


In [77]:
dfNbClicksByUser = dfClicks.groupby(by="user_id")['click_article_id'].count()

In [79]:
dfUser = pd.DataFrame(dfNbClicksByUser, columns=["click_article_id"])

In [81]:
dfUser[(dfUser.click_article_id > 10)]

,click_article_id
user_id,
1,12
3,17
5,87
6,35
7,22
...,...
321475,39
321650,12
321991,12


In [ ]:
min_book_ratings = 50
filter_books = df['ISBN'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['ISBN'].isin(filter_books)) & (df['userID'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

je veux un tableau 320k * 364k

memory based (kmeans)

In [51]:
users.shape

(2988181, 1)

In [17]:
# recommender.py

from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [44]:
dfClickSample = dfClicks[:10000]

In [45]:
from surprise import Reader
from surprise import KNNWithMeans
from surprise import Dataset

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).

data  = Dataset.load_from_df(dfClickSample[['user_id', 'click_article_id', 'session_size']], reader)
trainset = data.build_full_trainset()

In [48]:


sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

model based (svd)

In [49]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV


param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["rmse"])

2.122509817741076
1.2720904084592475
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
